## Import functions

In [1]:
from myfun import *

## Load the 'model'

In [5]:
# Model

granularite = 0.01

dic_of_words = LoadPickle("dic_of_words.p")
WordToRequest = LoadPickle("WordToRequest.p")
df = LoadPickle("df.p")
dic_of_requests = LoadPickle("dic_of_requests.p")

DefaultCategory = df.category.value_counts().index[0]
OverallDisDic = GetOverallCategoryDistribution(df)
Overallp =  DotProductFromDic(OverallDisDic)

PowerDicDis = GetPowerDicDis(Overallp,dic_of_words,granularite)

In [ ]:
## Most Powerful words by category

d = AutoVivification()

number_of_words = 0
for word,informations in dic_of_words.items():
    number_of_words = number_of_words + 1
    print("\rWords viewed so far : ",number_of_words,end = "")
    for category,frequence in informations.items():
        d[category][word] = d[category].get(word,0) + frequence
        
Powerful_Words_by_Category = {}
for category,informations in d.items():
    templist = [(word,frequence*PowerDicDis[word]) for word,frequence in informations.items()]
    total_mass = sum([item[1] for item in templist])
    templist = [(item[0],item[1],item[1]/total_mass) for item in templist]
    templist.sort(key = lambda a : -a[1])
    Powerful_Words_by_Category[category] = templist

## Functions used to explore

In [83]:
def Explore_Prediction(hyp,IdRequest):

    SelectedRequest, SelectedCategory, SelectedWords = GetInformationsFromIDRequest(IdRequest,df)
    UpdateDic(IdRequest,SelectedRequest, SelectedCategory, SelectedWords,dic_of_words,WordToRequest,True)
    sauvegardePowerDicDis = SavePowerDicDis(SelectedWords,PowerDicDis)
    UpdatePowerDicDis(SelectedWords,dic_of_words,sauvegardePowerDicDis,granularite,Overallp,PowerDicDis,True)

    RequestDic = GetRequestWeight(SelectedRequest,PowerDicDis)
    RequestDic_tuples = [(k,v) for k,v in RequestDic.items()]
    RequestDic_tuples.sort(key=lambda tup: -tup[1])
    ranked_words = [item[0] for item in RequestDic_tuples if item[1]>0.0001]
    ranked_words = Clean_Ranked_Words(ranked_words,WordToRequest)

    PREDICTIONS = ALL_PREDICTIONS_FROM_PARTITIONS(ranked_words,
                                                  RequestDic_tuples,dic_of_requests,WordToRequest,
                                                  hyp,[hyp],DefaultCategory,with_details=True)

    UpdateDic(IdRequest,SelectedRequest, SelectedCategory, SelectedWords,dic_of_words,WordToRequest,False)
    UpdatePowerDicDis(SelectedWords,dic_of_words,sauvegardePowerDicDis,granularite,Overallp,PowerDicDis,False)

    dftoprint = PREDICTIONS[0][2].head(25)
    dftoprint = dftoprint.drop(["score"], axis=1)
    atp = "ID : " + str(IdRequest) 
    btp = "Query : " + str(SelectedRequest)
    ctp = "Prediction : " + str(SelectedCategory==PREDICTIONS[0][1]).upper()
    dtp = "True Category : " + str(SelectedCategory) + " | Predicted Category : " + str(PREDICTIONS[0][1])    
    toprint = atp + "\n" + btp + "\n" + ctp + "\n" + dtp + "\n\nBest Queries Candidates"
    
    return toprint,dftoprint,SelectedCategory,PREDICTIONS[0][1]

In [84]:
def pick_queries_within_category(idcat,nobs=10):
    return df[df.category == idcat].head(nobs) 

## Explore the results

#### **Pick a random query and make a prediction**

In [187]:
idq = np.random.choice(np.arange(len(df)))
#idq = 57906
toprint,dftoprint,SelectedCategory,PredictedCategory = Explore_Prediction(0.85,idq)
print(toprint)
print(dftoprint)

ID : 294532
Query : shooting clothing uk
Prediction : FALSE
True Category : 829 | Predicted Category : 284

Best Queries Candidates
    idrequest                               request  category  score_relatif
0      370532                     shooting clothing       284       1.000000
1      566436                shooting clothing sale       829       1.000000
2      402918              ladies shooting clothing       829       1.000000
3      199680       pheasant shooting days for sale       284       0.894273
4      429568                      shooting vest uk       829       0.894273
5      602626  clay pigeon shooting gloucestershire       284       0.894273
6       20485                        shooting gilet       829       0.894273
7      306696                        shooting boots       829       0.894273
8       97802                 shooting sticks argos       829       0.894273
9      401423       lakenheath clay pigeon shooting       284       0.894273
10     202257        

#### **List the TOP 15 powerful words of the category the random request belongs to**

In [188]:
print("True Category : ", SelectedCategory)
Powerful_Words_by_Category[SelectedCategory][:15]

True Category :  829


[('shooting', 11252.904653119758, 0.06632920814943152),
 ('seeland', 6156.083740289631, 0.03628646757277881),
 ('shotgun', 3940.479277387829, 0.023226791504530952),
 ('realtree', 3892.012151618371, 0.02294110650384308),
 ('scopes', 3464.6438883625306, 0.02042202371021977),
 ('harkila', 3092.3695785958603, 0.01822768714180731),
 ('archery', 2786.3976494196504, 0.016424163837929908),
 ('bow', 2488.9412760446803, 0.014670834691975271),
 ('hunting', 2357.90877545604, 0.013898475707890674),
 ('crossbow', 2257.8454285206185, 0.013308661542428222),
 ('deerhunter', 2095.091764510158, 0.012349325087529924),
 ('recurve', 2060.289999530373, 0.012144189295086167),
 ('slingshot', 1955.5334894882571, 0.011526711712739005),
 ('ear', 1764.5191578502645, 0.010400795360179458),
 ('camo', 1659.9210387635985, 0.00978425139870414)]

#### **Pick some random queries from the category the random request belongs to**

In [189]:
print("True Category : ", SelectedCategory)
pick_queries_within_category(SelectedCategory,nobs=10)

True Category :  829


,idrequest,request,category
624,624,duck hunt,829
1084,1084,featherlight,829
1132,1132,ballistic gel,829
3003,3003,410 cartridges,829
5125,5125,spypoint,829
6389,6389,clay shooting,829
6883,6883,target shooting,829
8427,8427,harkila pro hunter,829
8968,8968,snare trap,829
10887,10887,mens hunters,829


#### **List the TOP 15 powerful words of the predicted category**

In [190]:
print("Predicted Category : ", PredictedCategory)
Powerful_Words_by_Category[PredictedCategory][:10]

Predicted Category :  284


[('shooting', 18218.988486003418, 0.09125580832253766),
 ('stag', 17696.53145033154, 0.0886389100715364),
 ('deer', 11145.859480615198, 0.05582771058526969),
 ('taxidermy', 9416.345404439711, 0.04716486933325116),
 ('clay', 9074.912159959951, 0.04545468840102631),
 ('pigeon', 4995.070494286749, 0.02501945681201686),
 ('duchally', 4990.948637194916, 0.024998811132318773),
 ('stalking', 4961.347488470459, 0.024850544023186934),
 ('archery', 4643.996082366084, 0.023260984915193805),
 ('harkila', 3607.764508361837, 0.01807067756263471)]

#### **Pick some random queries from the category the random request belongs to**

In [191]:
print("Predicted Category : ", PredictedCategory)
pick_queries_within_category(PredictedCategory,nobs=10)

Predicted Category :  284


,idrequest,request,category
606,606,stag head designs,284
3867,3867,wall mounted animal heads uk,284
4335,4335,shooting range las vegas,284
4358,4358,deer cushion,284
5317,5317,las vegas shooting range,284
8541,8541,baby banz ear defenders,284
8740,8740,colt 45 peacemaker,284
9424,9424,deerhunter trousers,284
10328,10328,uttings scopes,284
10603,10603,flash gun,284
